In [1]:
from __future__ import print_function
!nvidia-smi

Wed Jun 10 06:50:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 431.40       Driver Version: 431.40       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 166... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8     2W /  N/A |    153MiB /  6144MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import arithc as arith
import fqt, ppm
import contextlib, sys
import filecmp
from IPython.display import clear_output
import numpy as np

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D
#from tensorflow.keras.datasets import imdb
from tensorflow.keras.optimizers import Adam


with open('data\ecoli\Ecoli.txt') as f:
    for line in f:
        #inner_list = [elt.strip() for elt in line.split()]
        inner_list = list(line)
        ecoli = inner_list
print(len(ecoli)) # About 4 MB


temp_dict = {'a':97,'g': 103,'c': 99,'t': 116}
s =  [temp_dict[i] for i in ecoli]
char_list = [97, 103, 99, 116] # we can read this as we go
print(char_list)
update_period = len(s)


legend = dict([(v, k) for k, v in enumerate(char_list)]) # map character to 0,1,2,3,4, etc.
vocab_size = len(char_list)




n = 100000 # number of samples
tsteps = 20 #time steps
seg_len = 20 #input_dim
k = tsteps*seg_len # full context for each sample
n_symb = 256

# optimizer
sgd_opt = 'adam'
lr = 4e-3
beta1 = 0
beta2 = 0.9999
eps=1e-5

# LSTM Training
hidden_size = 90
batch_size = 16
epochs = 1

ln = True # Layer Nomalization
from tensorflow.keras.layers import LayerNormalization
fc = True # has fully connected layer
n_layer = 4 #only 4 total laters? or 4 LSTM it does say 4

opt = Adam(
    learning_rate=lr , beta_1=0.0, beta_2=beta2, epsilon=eps
)

4638690
[97, 103, 99, 116]


In [4]:
from plist import ProbabilityList
from tensorflow.keras.layers import TimeDistributed

s = s[:-40]
n_symb = 4
char_list = [97, 103, 99, 116]

max_features = 100
maxlen = k
embedding_size = 16

kernel_size = 3
filters = 32
# pool_size = 4

input_shape = (250,tsteps,seg_len)
NNPC3 = Sequential()
#NNPC3.add(TimeDistributed(Embedding(max_features, embedding_size, input_length=k)))
NNPC3.add(Conv1D(filters=filters,
                 kernel_size=3,
                 padding='valid',
                 activation='relu',
                 strides=1, input_shape = input_shape[1:]))
#NNPC3.add(TimeDistributed(MaxPooling1D(pool_size=pool_size)))
NNPC3.add(LSTM(352, activation='tanh', stateful=True, batch_input_shape=(250,tsteps-kernel_size+1,filters), return_sequences=True))
NNPC3.add(LayerNormalization(axis=1 , center=True , scale=True))
NNPC3.add(LSTM(352, activation='tanh', stateful=True, batch_input_shape=(250,tsteps-kernel_size+1,352), return_sequences=True))
NNPC3.add(LayerNormalization(axis=1 , center=True , scale=True))
NNPC3.add(LSTM(352, activation='tanh', stateful=True, batch_input_shape=(250,tsteps-kernel_size+1,352), return_sequences=True))
NNPC3.add(LayerNormalization(axis=1 , center=True , scale=True))
NNPC3.add(LSTM(352, activation='tanh', stateful=True, batch_input_shape=(250,tsteps-kernel_size+1,352)))
NNPC3.add(LayerNormalization(axis=1 , center=True , scale=True))
NNPC3.add(Dense(n_symb))
NNPC3.add(Activation('softmax'))
NNPC3.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['categorical_accuracy'])

inputfile, outputfile = 'data\ecoli\Ecoli.txt', 'data\ecoli\Ecoli_NNPC_PyExact.txt'

e_idx = 0
tempdict = {}
#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:

    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs) # For the first 200,000
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!
    for symbol in s[:n+k]:
        t = model.get_total() ## New lines!
        l = model.get_low(legend[symbol])
        h = model.get_high(legend[symbol])
        enc.storeRegion(l,h,t) 
        model.increment(legend[symbol])
        e_idx += 1
        
    prior = [1/257 for i in range(257)]
    prior[256] = 1-sum(prior[:256])

    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs)
        
    
    for overall in range(10):
        predicted_val = []
        if overall <= 8:
            x = np.zeros((500000, tsteps, seg_len)) # 64 characters context
            y = np.zeros((500000, n_symb))
            print(len(s))
            idx3 = 0
            for idx2 in range(500000*overall+k,500000*(overall+1)+k): #len(s)):
                train_seq = [legend[i] for i in s[idx2-k:idx2]] 
                train_target = legend[s[idx2]]
                x[idx3,:] = np.array(train_seq).reshape(tsteps,seg_len)
                y[idx3] = to_categorical(train_target, num_classes=n_symb )
                idx3 += 1
                
        if overall == 9:
            x = np.zeros((len(s)-500000*overall-k, tsteps, seg_len)) # 64 characters context
            y = np.zeros((len(s)-500000*overall-k, n_symb))
            print(len(s))
            idx3 = 0
            for idx2 in range(500000*overall+k,len(s)): #len(s)):
                train_seq = [legend[i] for i in s[idx2-k:idx2]] 
                train_target = legend[s[idx2]]
                x[idx3,:] = np.array(train_seq).reshape(tsteps,seg_len)
                y[idx3] = to_categorical(train_target, num_classes=n_symb )
                idx3 += 1

        if overall != 0:
            predicted_val += list(NNPC3.predict(x[0:n]))

        for i in range(len(x)//100000 -2):
            NNPC3.fit(x[n*i:n*(i+1)], y[n*i:n*(i+1)],
                  batch_size=250,
                  epochs=1,
                  validation_data=(x[n*(i+1):n*(i+2)], y[n*(i+1):n*(i+2)]))

            predicted_val += list(NNPC3.predict(x[n*(i+1):n*(i+2)]))
        i = len(x)//100000 -2
        NNPC3.fit(x[n*i:n*(i+1)], y[n*i:n*(i+1)],
                  batch_size=250,
                  epochs=1,
                  validation_data=(x[n*(i+1):], y[n*(i+1):]))
        predicted_val += list(NNPC3.predict(x[n*(i+1):]))

        NNPC3.fit(x[n*(i+1):], y[n*(i+1):],
                  batch_size=250,
                  epochs=1)
        x= None
        y = None
        del x
        del y
        for prob_list in predicted_val:
            for val, prob in enumerate(prob_list):
                model.set(val, int(prob*100000)+1)
            t = model.get_total()
            l = model.get_low(legend[s[e_idx]])
            h = model.get_high(legend[s[e_idx]])
            enc.storeRegion(l,h,t) 
            e_idx += 1
        predicted_val = None
        del predicted_val

    t = model.get_total()
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    
    enc.finish_encode(bitout)

ValueError: If a RNN is stateful, it needs to know its batch size. Specify the batch size of your input tensors: 
- If using a Sequential model, specify the batch size by passing a `batch_input_shape` argument to your first layer.
- If using the functional API, specify the batch size by passing a `batch_shape` argument to your Input layer.